In [51]:
from pathlib import Path
import czifile
import napari
import numpy as np
import skimage
from skimage import measure, exposure
from skimage.measure import regionprops_table
import pandas as pd
import pyclesperanto_prototype as cle
import matplotlib.pyplot as plt
# import gc

# Select a GPU with the following in the name. This will fallback to any other GPU if none with this name is found
cle.select_device("RTX")

c:\Users\albertds\mambaforge\envs\microglia\lib\site-packages\pyclesperanto_prototype\_tier0\_device.py:77: UserWarning: No OpenCL device found with RTX in their name. Using Intel(R) UHD Graphics 630 instead.
  warnings.warn(f"No OpenCL device found with {name} in their name. Using {device.name} instead.")


<Intel(R) UHD Graphics 630 on Platform: Intel(R) OpenCL HD Graphics (2 refs)>

In [52]:
directory_path = Path("./raw_data/")
images = []

# Iterate through the lsm files in the directory
for file_path in directory_path.glob("*.czi"):
    images.append(str(file_path))
    
images

['raw_data\\well 1.czi',
 'raw_data\\well 10.czi',
 'raw_data\\well 11.czi',
 'raw_data\\well 12.czi',
 'raw_data\\well 13.czi',
 'raw_data\\well 14.czi',
 'raw_data\\well 15.czi',
 'raw_data\\well 2.czi',
 'raw_data\\well 3.czi',
 'raw_data\\well 4.czi',
 'raw_data\\well 5.czi',
 'raw_data\\well 6.czi',
 'raw_data\\well 7.czi',
 'raw_data\\well 8.czi',
 'raw_data\\well 9.czi']

In [53]:
image = images[0]

# Extract filename and well_id
file_path = Path(image)
filename = file_path.stem
well_id = filename.split(" ")[1]

# Read the image file and remove singleton dimensions
img = czifile.imread(image)
img = img.squeeze()

# Image size reduction to improve processing times (slicing, not lossless compression)
slicing_factor = 4 # Use 2 or 4 for compression (None for lossless)

# Extract the stack containing the nuclei (0), dna_damage (1) and microglia channel (2)
ki67_stack = img[0, :, ::slicing_factor, ::slicing_factor]
nuclei_stack = img[2, :, ::slicing_factor, ::slicing_factor]
caspase_stack = img[3, :, ::slicing_factor, ::slicing_factor]

# Perform maximum intensity projections
ki67_mip = np.max(ki67_stack, axis = 0)
nuclei_mip = np.max(nuclei_stack, axis = 0)
caspase_mip = np.max(caspase_stack, axis = 0)

# Free up memory by deleting the img variable
del img
# Call the garbage collector
# gc.collect()

In [54]:
#Make 3D stack isotropic

# Voxel size from .czi file metadata
input_image = nuclei_stack

voxel_size_x = 0.21
voxel_size_y = 0.21
voxel_size_z = 2.17

# Adjust so voxel size_x and size_y so they are equal to 1 to avoid compression upon rescaling

multiplier = 0.5 / voxel_size_x

voxel_size_x = voxel_size_x * multiplier
voxel_size_y = voxel_size_y * multiplier
voxel_size_z = voxel_size_z * multiplier

print(voxel_size_x, voxel_size_y, voxel_size_z)

0.5 0.5 5.166666666666666


In [55]:
resampled = cle.scale(input_image, factor_x=voxel_size_x, factor_y=voxel_size_y, factor_z=voxel_size_z, auto_size=True)
background_subtracted = cle.top_hat_box(resampled, radius_x=5, radius_y=5, radius_z=5)
segmented = cle.voronoi_otsu_labeling(background_subtracted, spot_sigma=1, outline_sigma=1)
resampled_ki67 = cle.scale(ki67_stack, factor_x=voxel_size_x, factor_y=voxel_size_y, factor_z=voxel_size_z, auto_size=True)


In [56]:
nuclei_labels_3d = cle.pull(segmented)
resampled_ki67 = cle.pull(resampled_ki67)

# Extract regionprops
props = regionprops_table(label_image=nuclei_labels_3d, intensity_image=resampled_ki67, properties=["label", "intensity_mean", "intensity_max", "centroid", "area_filled"])

# Construct a dataframe
df = pd.DataFrame(props)

df

label  intensity_mean  intensity_max  centroid-0  centroid-1  \
0          1       43.373058          160.0   17.735751    7.170984   
1          2        4.534654           14.0   35.722772   12.623762   
2          3        5.379310           14.0   48.183908   17.632184   
3          4       16.657534           41.0   38.027397   31.698630   
4          5       10.569445           22.0   33.763889   33.069444   
...      ...             ...            ...         ...         ...   
17728  17729       58.610779          146.0   48.772455  447.017964   
17729  17730       15.034810           71.0   47.392405  452.686709   
17730  17731       27.000000           27.0  121.000000  469.000000   
17731  17732       50.569008          255.0   50.232446  470.915254   
17732  17733       19.792135           79.0   99.679775  474.859551   

       centroid-2  area_filled  
0        1.077720        193.0  
1        0.554455        101.0  
2        0.827586         87.0  
3        0.986301         73.0  
4        0.736111         72.0  
...           ...          ...  
17728  961.173653        167.0  
17729  961.212025        316.0  
17730  963.000000          1.0  
17731  960.690073        413.0  
17732  960.741573        178.0  

[17733 rows x 7 columns]

In [57]:
# Manually choose labels of cells undergoing mitosis to check the intensity_mean of Ki67 within those nuclei
labels_to_filter = [13731, 14156, 14854, 14874, 9636, 13134]

mitosis_df = df[df['label'].isin(labels_to_filter)]

mitosis_df

label  intensity_mean  intensity_max  centroid-0  centroid-1  \
9635    9636       74.154594          255.0   34.789694  494.500000   
13133  13134      105.701378          255.0   33.316998  443.588055   
13730  13731      103.920341          255.0   30.861017  374.584746   
14155  14156      103.858292          255.0   33.594203  380.294686   
14853  14854      103.785912          255.0   32.957182  394.265193   
14873  14874       70.172150          249.0   34.321519  487.918987   

       centroid-2  area_filled  
9635   462.938719        718.0  
13133  641.754977        653.0  
13730  676.300000        590.0  
14155  699.547504        621.0  
14853  740.983425        724.0  
14873  740.470886        395.0

In [58]:
# Manually choose labels of cells in G1_S phase to check the intensity_mean of Ki67 within those nuclei
labels_to_filter = [14384, 14468, 14656, 14558, 14756]

G1_S_df = df[df['label'].isin(labels_to_filter)]

G1_S_df


label  intensity_mean  intensity_max  centroid-0  centroid-1  \
14383  14384       42.863407          226.0   32.791980  382.895990   
14467  14468       39.250000          217.0   35.460526  386.996711   
14557  14558       22.142857           90.0   34.955150  431.264120   
14655  14656       21.512871           91.0   27.651485  429.374257   
14755  14756       26.803722          118.0   33.483926  426.255499   

       centroid-2  area_filled  
14383  710.442356        798.0  
14467  716.115132        304.0  
14557  722.774086        602.0  
14655  729.873267        505.0  
14755  733.208122        591.0

In [59]:
# Manually choose labels of cells under cell cyle arrest to check the intensity_mean of Ki67 within those nuclei
labels_to_filter = [14386, 14406, 14501, 14215, 14061, 14231]

G0_df = df[df['label'].isin(labels_to_filter)]

G0_df

label  intensity_mean  intensity_max  centroid-0  centroid-1  \
14060  14061        9.381742          127.0   33.122407  425.846473   
14214  14215       13.742358           76.0   33.168122  487.192140   
14230  14231        6.215686           29.0   34.081699  424.375817   
14385  14386        4.079890           16.0   34.644628  420.099174   
14405  14406        8.067164           33.0   33.690299  492.000000   
14500  14501        7.790393           27.0   32.862445  489.050218   

       centroid-2  area_filled  
14060  693.168050        482.0  
14214  701.054585        458.0  
14230  702.169935        306.0  
14385  712.151515        363.0  
14405  712.802239        268.0  
14500  718.727074        458.0

In [60]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(resampled)
viewer.add_image(resampled_ki67)
viewer.add_labels(nuclei_labels_3d)


<Labels layer 'nuclei_labels_3d' at 0x1435fbe2c10>

In [61]:
# Plot histogram
plt.figure(figsize=(10, 6))
plt.hist(df['intensity_mean'], bins=30, color='skyblue', edgecolor='black')
plt.xlabel('Intensity Mean')
plt.ylabel('Frequency')
plt.title('Histogram of Intensity Mean')
plt.show()